In [46]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import shutil

Extract the zip file:

In [47]:
class MyCLRuleMonitor(tf.keras.callbacks.Callback):
  def __init__(self, CL):
    super(MyCLRuleMonitor).__init__()
    self.CL = CL

  def on_epoch_end(self, epoch, logs=None):
    trainScore = logs["accuracy"]
    testScore = logs["val_accuracy"]

    if testScore > trainScore and testScore >= self.CL:
      self.model.stop_training = True

In [48]:
zip_name = 'fruits-small.zip'
extracted_file = 'fruits-small'

# Make new directory:
os.makedirs(extracted_file, exist_ok = True)

try:
    shutil.unpack_archive(filename= zip_name, extract_dir= extracted_file)
    print('Successfully extacted the file')
except Exception as e:
    print('Error while unpacking the zip file.')

Successfully extacted the file


Steps for Model creation:
0. Data Preprocessing
    i. Image Generators
    ii. Splitting Train and Test data from Generators
1. Model architecting + Prepro
    i. CNN + ANN layers creation
2. Model Compilation
3. Model Training 
4. Model Evaluation
5. Model Testing / Deploy

In [49]:
# Make data compatible using ImageDataGenerators:
# Each image in the Folders will be normalized using rescale.
train_image_generator= tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.0)
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1.0/255.0)

In [50]:
# Splitting Test and Train Data:

train_image_data = train_image_generator.flow_from_directory('fruits-small/fruits-small/data/Training',
                                                             batch_size= 20,
                                                             class_mode = 'categorical',
                                                             target_size = (64,64))


test_image_data = test_image_generator.flow_from_directory('fruits-small/fruits-small/data/Validation',
                                                           batch_size= 20,
                                                           class_mode = 'categorical',
                                                           target_size= (64,64))

Found 3425 images belonging to 7 classes.
Found 1150 images belonging to 7 classes.


In [51]:
train_image_data.image_shape
test_image_data.image_shape

(64, 64, 3)

In [52]:
# Model Architecting:
# CNN: A Convolution layer is combination of Convolve and Pooling:

model = tf.keras.Sequential()

# Conv2d(noofFeatureMaps, kernelShape, inputShape, activation) + padding = same -- add virtual boarder in every image
# NoOfFeatureMaps: Similar to no of units: Based on trial and error
# filter: (3,3), (4,4), (5,5) common :  in this content will be initialized randomly
# inputShape: similar to target_size

# ----------------- 1st convole layer:
model.add(tf.keras.layers.Conv2D(32, (3,3), input_shape= train_image_data.image_shape, activation='relu', padding= 'same'))
# Adding Pooling:
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

# ------------------- 2nd Convole layer:
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding= 'same'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# -------------------- Flatten:
model.add(tf.keras.layers.Flatten())

# FC Layer | ANN part, Layers creation:

model.add(tf.keras.layers.Dense(units= 128, activation='relu'))
model.add(tf.keras.layers.Dense(units= 128, activation='relu'))
model.add(tf.keras.layers.Dense(units= 256, activation= 'relu'))

# Output layer:
model.add(tf.keras.layers.Dense(units = 7, activation='softmax'))

# Model Summay:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,110,183 (4.24 MB)

 Trainable params: 1,110,183 (4.24 MB)

 Non-trainable params: 0 (0.00 B)

In [53]:
# COmpilation:
model.compile(optimizer= 'adam',
              loss= 'categorical_crossentropy',
              metrics= ['F1Score'])

In [58]:
# Model training:
# model.fit(train_image_data, 
#             validation_data = test_image_data,
#             steps_per_epoch= (len (train_image_data.filenames) // train_image_data.batch_size),
#             validation_steps=  (len(test_image_data.filenames) // test_image_data.batch_size),
#             epochs = 4,
#             callbacks= [MyCLRuleMonitor(0.9)])

model.fit(train_image_data, 
          validation_data = test_image_data, 
          epochs = 200, 
          steps_per_epoch = (len(train_image_data.filenames)//train_image_data.batch_size ),
          validation_steps= (len(test_image_data.filenames)//test_image_data.batch_size),
          callbacks= [MyCLRuleMonitor(0.9)])

Epoch 1/200
171/171 ━━━━━━━━━━━━━━━━━━━━ 15s 86ms/step - accuracy: 1.0000 - loss: 1.6973e-07 - val_accuracy: 0.9956 - val_loss: 0.0098
Epoch 2/200
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 1.0000 - loss: 4.7684e-08 - val_accuracy: 0.9956 - val_loss: 0.0095
Epoch 3/200
171/171 ━━━━━━━━━━━━━━━━━━━━ 12s 70ms/step - accuracy: 1.0000 - loss: 1.5429e-07 - val_accuracy: 0.9895 - val_loss: 0.0127
Epoch 4/200
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 1.6093e-07 - val_accuracy: 0.9904 - val_loss: 0.0128
Epoch 5/200
171/171 ━━━━━━━━━━━━━━━━━━━━ 12s 71ms/step - accuracy: 1.0000 - loss: 1.4000e-07 - val_accuracy: 0.9904 - val_loss: 0.0127
Epoch 6/200
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 5.9605e-09 - val_accuracy: 0.9912 - val_loss: 0.0123
Epoch 7/200
171/171 ━━━━━━━━━━━━━━━━━━━━ 15s 87ms/step - accuracy: 1.0000 - loss: 1.2597e-07 - val_accuracy: 0.9877 - val_loss: 0.0145
Epoch 8/200
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accur

In [ ]:
# Input / Deployment / Testing:

image = tf.keras.preprocessing.image.load_img('flowers-fruits-garden-strawberry-plant-species.jpg', target_size = (64,64))
image_array = tf.keras.preprocessing.image.img_to_array(image)

image_np_array = np.expand_dims(image_array, axis = 0)
prediction_probabilities = model.predict(image_np_array)

train_dir = 'fruits-small/fruits-small/data/Training'
class_names = sorted(os.listdir(train_dir))

prediction_class_index= np.argmax(prediction_probabilities)
prediction_class_name = class_names[prediction_class_index]
print(prediction_class_index)
print(class_names)
print(prediction_class_name)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1
['Banana', 'Lemon', 'Mango', 'Orange', 'Pineapple', 'Pomegranate', 'Strawberry']
Lemon
